# Week3_4 Assignment

## [BASIC](#Basic) 
- Encoder & Decoder Layer 코드를 직접 필사하고 각 함수에 주석을 달 수 있다. 

## [CHALLENGE](#Challenge)
- 텐서의 크기(shape)를 계산할 수 있다. 

## [ADVANCED](#Advanced)
- 완성된 transformer 모델의 모든 학습 가능한 파라미터 이름과 크기(shape)를 출력할 수 있다.

### Informs
이번 과제에서는 "[Annotated Transformer](https://nlp.seas.harvard.edu/2018/04/03/attention.html)"의 코드를 필사해본다.   
"Annotated Transformer"는 "Attention is all you need" 논문에서 제안한 transformer 모델을 pytorch 라이브러리로 직접 구현한다.   
코드 필사를 통해 다음을 배울 수 있다.    
- Encoder, Decoder 구조
- Attention Mechanism
- "residual connection", "layer normalization" 등의 구조 

코드 필사를 시작하기 앞서, transformer 모델의 최종 구조를 살펴보자.    

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_full.png?raw=true" width="500" align="center"/>

최종 모델은 `EncoderDecoder()` 클래스에 여러 인스턴스를 생성자의 입력 파라미터로 넣어 생성한다.    
앞으로 우리는 `EncoderDecoder()` 클래스와 같은 여러 클래스들을 구현하고 연결할 것이다. 따라서 대략적인 클래스간의 관계를 살펴보고 이해한다면 보다 큰 그림을 가지고 코드 필사를 할 수 있을 것이다. 

Transformer 모델은 크게 4가지 클래스로 구현된다.    
- Frame
    - frame 역할을 하는 `EncoderDecoder` 클래스
- Input Embedding & Encoding
    - 입력값을 벡터화하는 `Embeddings`, `PositionalEncoding`
- Encoder & Decoder
    - 각 6개 layer를 갖고 있는 `Encoder`, `Decoder`
    - layer 1층을 구현한 `EncoderLayer`, `DecoderLayer`
- Sublayer
    - `EncoderLayer`, `DecoderLayer` 내부에서 사용되는 Sublayer 클래스인 `MultiHeadAttiontion`, `PositionwiseFeedForward`
    - Sublayer 클래스들을 연결하는 `SublayerConnection`
    
아래 좌측 도식에서 각 클래스의 색상은 아래 우측 도식(transformer 구조)의 색상과 맵핑되어 있다.    
각 클래스의 역할과 클래스 간 연결 관계를 생각하면서 transformer를 코드로 구현해보자.   


<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_map.png?raw=true" width="400" height="400" align="left"/>
<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_transformer.png?raw=true" width="300" height="400" align="right"/>



In [ ]:
import os
import sys
import pandas as pd
import numpy as np 

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

import math, copy, time
import random

## Basic

### Frame
- `EncoderDecoder`

아래 도식은 `EncoderDecoder` 클래스의 `forward()`, `encode()`, `decode()` 메소드를 도식화 한 것이다.    
 
<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_encoderdecoder.png?raw=true" width=500>


- `Generator`

In [ ]:
class EncoderDecoder(nn.Module):
    """
    A standard Encoder-Decoder architecture.
    Base for this and many models.
    """
    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
      super(EncoderDecoder, self).__init__()
      self.encoder = encoder
      self.decoder = decoder
      self.src_embed = src_embed
      self.tgt_embed = tgt_embed
      self.generator = generator    
    
    def forward(self, src, tgt, src_mask, tgt_mask): # src = source
      "src와 tgt입력을 각각 encoder, decoder 처리해 그 결과를 반환한다"
      return self.decode(
          self.encode(src, src_mask), src_mask,
          tgt, tgt_mask)
    
    
    def encode(self, src, src_mask):
      return self.encoder(self.src_embed(src), src_mask)
    
    
    def decode(self, memory, src_mask, tgt, tgt_mask):
      return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)

In [ ]:
class Generator(nn.Module):
    
    "vocab(총 단어의 수)에서 하나의 단어를 예측하는 linear + softmax 모델 정의"

    def __init__(self, d_model, vocab):
      super(Generator, self).__init__()
      self.proj = nn.Linear(d_model, vocab)
    
    def forward(self, x):
      return F.log_softmax(self.proj(x), dim=-1)

### Encoder
The encoder is composed of a stack of 
`N=6`
 identical layers.
- `Encoder`
- `EncoderLayer`
- `SublayerConnection`
- Reference
    - Layer Normalization
        - [한국어 설명](https://yonghyuc.wordpress.com/2020/03/04/batch-norm-vs-layer-norm/)
        - [torch official docs](https://pytorch.org/docs/stable/generated/torch.nn.LayerNorm.html)
    - Residual Connection
        - [한국어 설명](https://itrepo.tistory.com/36)
    - pytorch ModuleList
        - [torch official docs](https://pytorch.org/docs/1.9.1/generated/torch.nn.ModuleList.html)


In [ ]:
def clones(module, N):
  "module과 동일한 구조의 레이어를 N개 만들어 ModuleList에 담아 반환"
  return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

In [ ]:
class Encoder(nn.Module):
    "Core encder : N개의 EncoderLayer를 쌓은 모델"
    def __init__(self, layer, N):
      super(Encoder,self).__init__()
      self.layers = clones(layer, N)
      self.norm = LayerNorm(layer.size)    
    
    def forward(self, x, mask):
      
      "input 값인 x, mask를 순차적으로 EncoderLayer에 입력"

      for layer in self.layers:
        x = layer(x, mask)
      return self.norm(x) # 레이어 정규화

- 두 sub-layer 각각에 `residual connection`을 적용하고, 이후에 `layer normalization(레이어 정규화)`을 진행한다.

In [ ]:
class LayerNorm(nn.Module):
    
    "Layer normalization : 각 입력값의 features를 정규화"

    def __init__(self, features, eps=1e-6):
      super(LayerNorm, self).__init__()
      self.a_2 = nn.Parameter(torch.ones(features))
      self.b_2 = nn.Parameter(torch.zeros(features))
      self.eps = eps
    
    def forward(self, x):
      mean = x.mean(-1, keepdim=True)
      std = x.std(-1, keepdim=True)
      return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

- 각 sub-layer의 output은 `LayerNorm(x + sublayer(x))` 이다.
  - 여기서 `sublayer(x)`는 sub-layer 자체에서 수행되는 기능이다.
- 각 sub-layer output이 sub-layer의 input으로 더해지고 normalize되기 전에, 여기에 **`dropout`**을 적용한다.
- `Residual connection`을 적용하기 위해서
  - embedding layer들을 포함한 모델 내 모든 sub-layer들은 `d_model=512` 차원의 output을 생성한다.



In [ ]:
class SublayerConnection(nn.Module):
    """
    입력값을 순차적으로
    1. layer norm
    2. sublayer
    3. dropout
    4. residual connection
    에 통과시킨다.
    """
    def __init__(self, size, dropout):
      super(SublayerConnection, self).__init__()
      self.norm = LayerNorm(size)
      self.dropout = nn.Dropout(dropout)
    
    def forward(self, x, sublayer):
      "sublayer에 같은 크기의 residual connection을 적용"
      return x + self.dropout(sublayer(self.norm(x)))

각각의 layer는 두개의 sub-layers를 갖고 있다. 
- 첫번째 sub-layer<br>
  : `multi-head self-attention layer`
- 두번째 sub-layer<br>
  : `position-wise fully connected feed-forward network`

In [ ]:
class EncoderLayer(nn.Module):
    "self-attn(attention),feed forward (defined below) 총 2개의 sublayer로 구성된 인코더"
    def __init__(self, size, self_attn, feed_forward, dropout):
      super(EncoderLayer, self).__init__()

      # self_attn ; MultiHeadAttention
      self.self_attn = self_attn

      # feed_forward : PositionwiseFeedForward
      self.feed_forward = feed_forward

      # SublayerConnection 2개 : #1 for self-attn layer, 1 for ff layer
      self.sublayer = clones(SublayerConnection(size, dropout), 2) 

      # d_model : 임베딩 층에서 input 단어를 output 벡터로 변환하는데, 이 때 output 벡터의 길이를 가리킨다.
      self.size = size

        
    def forward(self, x, mask):
      x = self.sublayer[0](x, lambda x : self.self_attn(x, x, x, mask))

      return self.sublayer[1](x, self.feed_forward)

### Decoder
he decoder is also composed of a stack of 
`N=6`
 identical layers.
- `Decoder`
- `DecoderLayer`

In [ ]:
class Decoder(nn.Module):
    "N개의 DecoderLayer를 쌓은 모델"
    def __init__(self, layer, N):
      super(Decoder, self).__init__()
      self.layers = clones(layer, N)
      self.norm = LayerNorm(layer.size)
    
    def forward(self, x, memory, src_mask, tgt_mask):
    " 입력값 x, tgt_mask와 encoder에서 전달 받은 memory, src_mask를 순차적으로 DecoderLayer에 입력 "
      for layer in self.layers:
        x = layer(x, memory, src_mask, tgt_mask)
      return self.norm(x)

- 디코더의 third sub-layer
  - encoder stack의 output에 대한 multi-head attention을 수행
  - 인코더와 비슷하게 각각의 sub-layer들에 `residual connections`를 적용하고,<br> 그 후에 레이어 정규화를 진행한다.

  - cf. 2개의 sub-layer가 있던 인코더 파트

In [ ]:
class DecoderLayer(nn.Module):
    "self-attn, src-attn, feed forward (defined below) 총 3개의 sublayer로 구성된 디코더"
    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
      super(DecoderLayer, self).__init__()
      self.size = size
      self.self_attn = self_attn
      self.src_attn = src_attn
      self.feed_forward = feed_forward
      self.sublayer = clones(SublayerConnection(size, dropout), 3)
    
    def forward(self, x, memory, src_mask, tgt_mask):

      m = memory # 인코더로부터 넘어온 인코딩 (n_batch, n_seq, d_model)
      x = self.sublayer[0](x, lambda x : self.self_attn(x,x,x,tgt_mask))
      x = self.sublayer[1](x, lambda x : self.src_attn(x,x,x,src_mask))
      return self.sublayer[2](x, self.feed_forward)

### Sublayer
- `attention` 함수

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_attention.png?raw=true" width="500" align="center"/>  

- `MultiHeadedAttention`
- `PositionwiseFeedForward`

### Challenge


### Q1. 위 도식에 따라 `score`, `p_attn`, `attention` 을 구하라 

In [ ]:
def attention(query, key, value, mask=None, dropout=None):
  "Compute 'Scaled Dot Product Attention'"
  # 아래서 nbatches는 미니배치 크기, 코드에서 nbatches라는 변수명을 쓰므로
  # 표기를 통일하기 위해 nbatches로 표기
  # query: (nbatches, h, n_seq, d_k)
  # key:   (nbatches, h, n_seq, d_k)
  # value: (nbatches, h, n_seq, d_v) 인데 d_k=d_v로 두었음
  # 이 함수는 아래쪽 MultiHeadedAttention 클래스의 foward 함수에서 호출됨

  d_k = query.size(-1) # query의 last dimension


  scores = torch.matmul(query, key.transpose(-2,-1)) / math.sqrt(d_k)
 # scores : (nbatches, h, n_seq, n_seq)

  if mask is not None:
    scores = scores.masked_fill(mask ==0, -1e9)

  p_attn = F.softmax(scores, dim = -1)

  if dropout is not None:
    p_attn = dropout(p_attn)
  
  # torch.matmul(p_attn, value): (nbatches, h, n_seq, n_seq)*(nbatches, h, n_seq, d_v)
    # = (nbatches, h, n_seq, d_v),      p_attn: (nbatches, h, n_seq, nseq)
  return torch.matmul(p_attn, value), p_attn

###Q2. query, key, value가 모두 (m, d_k) shape의 matrix라고 가정할 때, `score`, `p_attn`, `attention`의 shape을 각각 구하라
- score : (m, m)
- p_attn : (m, m)
- attention : (m, d_k)

### (아래의 **Q3을 먼저 풀고 돌아오세요**) Q4.  query, key, value가 모두 (12, 8, 1, 64) shape의 tensor라고 가정할 때 , `score`, `p_attn`, `attention`의 shape을 각각 구하라

- score : (12, 8, 1, 1)
- p_attn : (12, 8 1, 1)
- attention : (12, 8, 1, 64)


- `MultiHeadedAttention`

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_multihead.png?raw=true" width="300" align="center"/>  

In [ ]:
class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
      "Take in model size and number of heads"
      super(MultiHeadedAttention, self).__init__()
      assert d_model % h == 0
      # We assume d_v always equals d_k
      self.d_k = d_model // h # Q, K의 차원, multi headed attention은 d_model을 head 개수로 나눠 dimesion reduction을 함.
      self.h = h

      self.linears = clones(nn.Linear(d_model, d_model), 4)


      self.attn = None
      self.dropout = nn.Dropout(p=dropout)
    
    
    def forward(self, query, key, value, mask=None):
      "Implements Figure 2"
      if mask is not None:
        mask = mask.unsqeeze(1) # 새로운 dimension을 (axis 1 앞에) 추가
      nbatches = query.size(0)  # size(0)을 하면 Size(x,y)중 x를 리턴함

      # 1) Do all the linear projections in batch from d_model => h x d_k
      query, key, value = \
          [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1,2)
          for l, x in zip(self.linears, (query, key, value))]
       # 이 라인이 실행되면 query, key, value는 각각 (nbatches, h, n_seq, d_k) 가 된다.

      # 2) Apply attention on all the projected vectors in batch
      x, self.attn = attention(query, key, value, mask=mask,dropout=self.dropout)
      # x: (nbatches, h, n_seq, d_v),  self.attn: (nbatches, h, n_seq, n_seq)
      
      # 3) "Concat" using a view and apply a final Linear.
      # 8개 head의 attention을 concatenate하여 마지막 linear layer에 통과
      x = x.transpose(1,2).contiguous().view(nbatches, -1, self.h * self.d_k)
      # x: (nbatches, n_seq, h*d_k) 여기서 h*d_k=d_model

      # 4) matmul x and Wo -> (nbatches, n_seq, d_model)
      return self.linears[-1](x)

      
            

### Q3.  query, key, value가 모두 (12, 512) shape의 matrix이고, h 값이 8 이라고 가정할 때, 아래 값의 shape을 각각 구하라

- `d_k` (d_k = d_model // h) : 512/8 = 64
- `nn.Linear(d_model, d_model)(query)` : (12, 512)
- `nn.Linear(d_model, d_model)(query).view(nbatches, -1, h, d_k)` : (12, 1, 8, 64)
- `nn.Linear(d_model, d_model)(query).view(nbatches, -1, h, d_k).transpose(1,2)` : (nbatches, h, m, d_k)  = (12, 8, 1, 64)

- `PositionwiseFeedForward`

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_pwff.png?raw=true" width="300" align="center"/>  

In [ ]:
class PositionwiseFeedForward(nn.Module):

  def __init__(self, d_model, d_ff, dropout=0.1):
    super(PositionwiseFeedForward, self).__init__()
    self.w_1 = nn.Linear(d_model, d_ff)
    self.w_2 = nn.Linear(d_ff, d_model)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    return self.w_2(self.dropout(F.relu(self.w_1(x))))

### Input Embedding & Encoding
- `Embeddings`
    - [pytorch official docs](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html)

In [ ]:
class Embeddings(nn.Module):
    '''
    입력값 x (연속적인 토큰)를 지정된 vocab 사이즈의 lookup 테이블에서 d_model 사이즈의 엠베딩으로 변환
    '''
    def __init__(self, d_model, vocab):
      super(Embeddings, self).__init__()
      self.lut = nn.Embedding(vocab, d_model)
      self.d_model = d_model
    
    def forward(self, x):
      # 임베딩 벡터에 sqrt(d_model)을 곱해서 임베딩 벡터의 요소들 값을 
      # 증가 시킴.
      # d_model은 512정도 되는 큰 값이이므로 22정도 되는 값이 
      # 임베딩 벡터 요소에 곱해짐. 
      # 곱하는 이유는 뒤에 포지션 벡터를 더할텐데 이 때 포지션 벡터에
      # 의해 임베딩 결과가 희석되는 것을 막기 위함
      # ref.: https://stackoverflow.com/questions/56930821/why-does-embedding-vector-multiplied-by-a-constant-in-transformer-model
      return self.lut(x) * math.sqrt(self.d_model)

- `PositionalEncoding`

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_pe.png?raw=true" width="500" align="center"/>  

- `position` 변수 설명
    - 모든 position (=최대 토큰 개수)의 값을 갖고 있는 matrix
- `div_term` 변수 설명

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_div.png?raw=true" width="500" align="center"/>  
- `Embedding` + `Encoding` 도식화 

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_emb_enc.png?raw=true" width="400" align="center"/>  


In [ ]:
class PositionalEncoding(nn.Module):
    ' 입력값 x (엠베딩 된 3차원 텐서 (nbatches, max_len, d_model))에 positional encoding을 더해 반환'
    def __init__(self, d_model, dropout, max_len = 5000):
      super(PositionalEncoding, self).__init__()
      self.dropout = nn.Dropout(p=dropout)

      # Compute the positional encodings once in log space
      pe = torch.zeros(max_len, d_model) # Size: (max_len, d_model)
      position = torch.arange(0, max_len).unsqueeze(1) #torch.arange(start, end, step), #unsqueeze(1):두번째 차원에 1 추가
      div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
      pe[:, 0::2] = torch.sin(position * div_term)
      pe[:, 1::2] = torch.cos(position * div_term)
      pe = pe.unsqueeze(0)
      self.register_buffer('pe',pe)

    def forward(self, x):
      x = x + Variable(self.pe[:, :x.size(1)],
                       requires_grad=False)
      return self.dropout(x)

### Q4.  max_len이 512이고, d_model이 512라고 가정할 때, `position`과 `div_term`의 shape을 구하라

- `position` : (512, 1)
- `div_term` : (256)
- `position * div_term` : (512, 256)

### Advanced

### Finally Build Model
- Xavier Initialization
    - [한국어 자료](https://huangdi.tistory.com/8)
    - [pytorch official docs](https://pytorch.org/docs/stable/nn.init.html#torch.nn.init.xavier_uniform_)

In [ ]:
def make_model(src_vocab, tgt_vocab, 
               N=6, d_model=512, d_ff=2048, h=8, dropout=0.1):
  "Helper: Construct a model from hyperparameters."
  '''
  src_vocab: 입력을 임베딩할 때 사용하는 단어장 사이즈
  tgt_vocab: 출력을 위한 출력쪽 단어장 사이즈
  d_model: 트랜스포머 인코더 디코더에서 사용되는 벡터의 크기
  d_ff: feed foward층이 출력하는 벡터의 크기
  '''
  c = copy.deepcopy
  attn = MultiHeadedAttention(h, d_model)
  ff = PositionwiseFeedForward(d_model, d_ff, dropout)
  position = PositionalEncoding(d_model, dropout)
  model = EncoderDecoder(
      Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
      Decoder(DecoderLayer(d_model, c(attn), c(attn), 
                            c(ff), dropout), N),
      nn.Sequential(Embeddings(d_model, src_vocab), c(position)),
      nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)),
      Generator(d_model, tgt_vocab))
  
  # This was important from their code. 
  # Initialize parameters with Glorot / fan_avg.
  # 모든 학습 파라미터를 
  for p in model.parameters():
      if p.dim() > 1:
          nn.init.xavier_uniform_(p)
  return model
            

In [ ]:
model = make_model(10,10)

### Q5. 위 코드로 만든 모델의 모든 파라미터의 이름과 크기 (shape) 을 출력하라

In [ ]:
# 구현
for name, param in model.named_parameters():
  print(f'name:{name}') 
  print(type(param)) 
  print(f'param.shape:{param.shape}')
  print('=====')


name:encoder.layers.0.self_attn.linears.0.weight
<class 'torch.nn.parameter.Parameter'>
param.shape:torch.Size([512, 512])
=====
name:encoder.layers.0.self_attn.linears.0.bias
<class 'torch.nn.parameter.Parameter'>
param.shape:torch.Size([512])
=====
name:encoder.layers.0.self_attn.linears.1.weight
<class 'torch.nn.parameter.Parameter'>
param.shape:torch.Size([512, 512])
=====
name:encoder.layers.0.self_attn.linears.1.bias
<class 'torch.nn.parameter.Parameter'>
param.shape:torch.Size([512])
=====
name:encoder.layers.0.self_attn.linears.2.weight
<class 'torch.nn.parameter.Parameter'>
param.shape:torch.Size([512, 512])
=====
name:encoder.layers.0.self_attn.linears.2.bias
<class 'torch.nn.parameter.Parameter'>
param.shape:torch.Size([512])
=====
name:encoder.layers.0.self_attn.linears.3.weight
<class 'torch.nn.parameter.Parameter'>
param.shape:torch.Size([512, 512])
=====
name:encoder.layers.0.self_attn.linears.3.bias
<class 'torch.nn.parameter.Parameter'>
param.shape:torch.Size([512])
==